In [1]:
# Requirements - uncomment as needed
#!pip install matplotlib numpy scipy phyre

# Generating the dataset

The requirement for the dataset here: 

- only using the template that contains only two balls on the scene;

- red ball must be in the free-fall without any collisions other than with the floor;

- only coordinates of the red ball are recorded.

## Playground 2 + Generation

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import re
import itertools

import phyre

In [2]:
# Choosing a setup where only one ball is needed
eval_setup = 'ball_cross_template'


# We only need one fold as we mix all the data together anyway
fold_id = 0

train_tasks, dev_tasks, test_tasks = phyre.get_fold(eval_setup, fold_id)
tasks = list(train_tasks + dev_tasks + test_tasks)

print('Total tasks: ', len(tasks))

Total tasks:  2500


In [9]:
# Filtering tasks to only include a simple two-ball template. The template key: '00000:xxx'

task_filter = re.compile("00000:*")
tasks = list(filter(task_filter.match, tasks))

print('Tasks after filtering: ', len(tasks))

# Choosing a single scenario in which we will generate our free-falls
tasks = [tasks[0]]

Tasks after filtering:  100


In [10]:
# Getting action tier for our tasks - a single ball
action_tier = phyre.eval_setup_to_action_tier(eval_setup)
print('Action tier for', eval_setup, 'is', action_tier)

Action tier for ball_cross_template is ball


In [11]:
# Create the simulator from the tasks and tier.
simulator = phyre.initialize_simulator(tasks, action_tier)

In [6]:
# getting a 10000 actions from a simulator
# it uniformly samples actions skipping invalid ones
# Action dimensions: 3 (x, y, radius) - represent coordinates and size of the red ball

actions = simulator.build_discrete_action_space(max_actions=10000)

In [7]:
# Defining a function to check if the red ball is in the free fall throughout the simulation
def is_red_ball_in_free_fall(simulation):
    features = simulation.featurized_objects.features
    return False not in [features[0][-1][0] == features[frame_id][-1][0] for frame_id in range(len(features))]

In [8]:
# Getting only the coordinates of the red ball
def get_red_ball_data(simulation):
    features = simulation.featurized_objects.features
    data = []
    for frame_id in range(len(features)):
        data.append([features[frame_id][-1][0], features[frame_id][-1][1]])
    return data

In [18]:
# we are using a single task
task_index = 0

for action_index in range(len(actions)):
    simulation = simulator.simulate_action(task_index, actions[action_index], need_images=True, need_featurized_objects=True, stride=15)
    if simulation.status.is_invalid(): continue
    if is_red_ball_in_free_fall(simulation):
        np.save(f'data-free-fall/task-{task_index}-action-{action_index}', get_red_ball_data(simulation))

In [19]:
# DO NOT INCLUDE IT IN THE SCRIPT
# Checking what is in one of the files that I saved
np.load('data-free-fall/task-0-action-1.npy')

array([[0.30078125, 0.14446744],
       [0.30078125, 0.13585415],
       [0.30078125, 0.1128854 ],
       [0.30078125, 0.07556117],
       [0.30078125, 0.02388148],
       [0.30078125, 0.02498015],
       [0.30078125, 0.0200189 ],
       [0.30078125, 0.01964036],
       [0.30078125, 0.01964815],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.30078125, 0.01964832],
       [0.

In [21]:
3/2*0.07556117 - 1/2*0.13585415

0.045414679999999985